In [2]:
import time
import numpy as np
import generate_data as data
import algorithms as test



print('step1')
### Pipeline parameters


test.load_data(10, 3, 4)
print('dataset after', test.dataset_modified)
print('queries after', test.queries_modified )
methods = ['flowtree']
ncs = [1]

# Parameter for Sinkhorn or LC-WMD
method_param = 1


target_accuracy = 0.9
sort_flag = True

### Query set

query_idx = np.arange(1)
qn = len(query_idx)
queries = test.queries[query_idx]
queries_modified = [test.queries_modified[i] for i in query_idx]
answers = test.answers[:,query_idx]


step1
generation step 0
generation step 1
generation step 2
generation step 3
generation step 4
generation step 5
generation step 6
dataset after [[(0, 0.3333333333333333), (1, 0.3333333333333333), (2, 0.3333333333333333)], [(0, 0.3333333333333333), (1, 0.3333333333333333), (3, 0.3333333333333333)], [(0, 0.3333333333333333), (1, 0.3333333333333333), (4, 0.3333333333333333)], [(0, 0.3333333333333333), (1, 0.3333333333333333), (5, 0.3333333333333333)], [(0, 0.3333333333333333), (1, 0.3333333333333333), (6, 0.3333333333333333)], [(0, 0.3333333333333333), (1, 0.3333333333333333), (7, 0.3333333333333333)], [(0, 0.3333333333333333), (1, 0.3333333333333333), (8, 0.3333333333333333)], [(0, 0.3333333333333333), (1, 0.3333333333333333), (9, 0.3333333333333333)], [(0, 0.3333333333333333), (2, 0.3333333333333333), (3, 0.3333333333333333)], [(0, 0.3333333333333333), (2, 0.3333333333333333), (4, 0.3333333333333333)], [(0, 0.3333333333333333), (2, 0.3333333333333333), (5, 0.3333333333333333)], [(0, 0

In [3]:
def call_method_weighted(fnc, q_index, input_idx, nc):
    result = np.zeros(nc, dtype=np.int32)
    score_result = np.zeros(nc, dtype=np.float32)
    fnc(queries_modified[q_index], input_idx, result, score_result, sort_flag)
    return result

def call_method_uniform(fnc, q_index, input_idx, nc, method_param=-1):
    result = np.zeros(nc, dtype=np.int32)
    score_result = np.zeros(nc, dtype=np.float32)
    if method_param == -1:
        fnc(queries[q_index], input_idx, result, score_result, sort_flag)
    else:
        fnc(queries[q_index], input_idx, result, score_result, sort_flag, method_param)
    return result

def call_exact(q_index, input_idx):
    return [test.exact_emd(queries[q_index], input_idx)]

In [3]:
fdic = {}
fdic["mean"] = lambda q_index, input_idx, nc:call_method_weighted(test.solver.means_rank, q_index, input_idx, nc)
fdic["overlap"] = lambda q_index, input_idx, nc:call_method_weighted(test.solver.overlap_rank, q_index, input_idx, nc)
fdic["quadtree"] = lambda q_index, input_idx, nc:call_method_weighted(test.solver.quadtree_rank, q_index,input_idx,  nc)
fdic["flowtree"] = lambda q_index, input_idx, nc:call_method_weighted(test.solver.flowtree_rank, q_index, input_idx, nc)
fdic["rwmd"] = lambda q_index, input_idx, nc:call_method_uniform(test.rwmd, q_index, input_idx, nc)
fdic["lcwmd"] = lambda q_index, input_idx, nc:call_method_uniform(test.lc_wmd, q_index, input_idx, nc, method_param)
fdic["sinkhorn"] = lambda q_index, input_idx, nc:call_method_uniform(test.sinkhorn, q_index, input_idx, nc, method_param)
fdic["exact"] = lambda q_index, input_idx, nc:call_exact(q_index, input_idx)

In [4]:
print(test.solver.dataset_embedding)

[[(0, 1.0), (1, 0.16666666666666666), (2, 0.08333333333333333), (3, 0.041666666666666664), (4, 0.020833333333333332), (5, 0.010416666666666666), (6, 0.005208333333333333), (7, 0.0026041666666666665), (8, 0.0013020833333333333), (9, 0.0006510416666666666), (10, 0.0003255208333333333), (11, 0.00016276041666666666), (12, 8.138020833333333e-05), (13, 4.0690104166666664e-05), (14, 2.0345052083333332e-05), (15, 1.0172526041666666e-05), (16, 5.086263020833333e-06), (17, 2.5431315104166665e-06), (18, 1.2715657552083333e-06), (19, 6.357828776041666e-07), (21, 0.16666666666666666), (22, 0.08333333333333333), (23, 0.041666666666666664), (24, 0.020833333333333332), (25, 0.010416666666666666), (26, 0.005208333333333333), (27, 0.0026041666666666665), (28, 0.0013020833333333333), (29, 0.0006510416666666666), (30, 0.0003255208333333333), (31, 0.00016276041666666666), (32, 8.138020833333333e-05), (33, 4.0690104166666664e-05), (34, 2.0345052083333332e-05), (35, 1.0172526041666666e-05), (36, 5.0862630208

In [5]:
### Main

accuracy = 0
accs = np.zeros(len(methods))
total_time = 0
clength = 0
input_idx = None
orig_input_idx = np.zeros(len(test.dataset), dtype=np.int32)
for i in range(len(test.dataset)):
    orig_input_idx[i] = i

start = time.time()
for q in range(qn):
    input_idx = orig_input_idx
    for m in range(len(methods)):
        input_idx = fdic[methods[m]](q, input_idx, ncs[m])
        if m==0:
            clength += np.mean([len(test.dataset[j]) for j in input_idx])
        if answers[0,q] in input_idx:
            accs[m] += 1
    if answers[0,q] in input_idx:
        accuracy += 1
    total_time = time.time() - start
    # At the end of each iteration of the main loop, input_idx is a list of indices (in the dataset) of the final candidates
    # for the current query, of the length specified by the final number in the input list of candidate numbers (third parameter).
    # As an example let us print the top-10 nearest neighbors found by the pipeline for each of the first three queries:
    if q < 3:
        print("Nearest neighbors found by pipeline for query " + str(q) + ": " + str(input_idx))

print("=== Output ===")
print("Total time: ", total_time*1./qn)
print("Final accuracy:", accuracy*1./qn)
print("Accuracy after each stage of the pipeline:", [acc*1./qn for acc in accs])

Nearest neighbors found by pipeline for query 0: [4]
=== Output ===
Total time:  0.014467000961303711
Final accuracy: 1.0
Accuracy after each stage of the pipeline: [1.0]


In [4]:
accuracies =[]
for epoch in range(100):
    print(epoch)
    test.load_data(20, 5, 3)
    print('step 1')
    methods = ['quadtree']
    ncs = [1]

    # Parameter for Sinkhorn or LC-WMD
    method_param = 1


    target_accuracy = 0.9
    sort_flag = True

    ### Query set

    query_idx = np.arange(1)
    qn = len(query_idx)
    queries = test.queries[query_idx]
    queries_modified = [test.queries_modified[i] for i in query_idx]
    answers = test.answers[:,query_idx]

    fdic = {}
    fdic["mean"] = lambda q_index, input_idx, nc:call_method_weighted(test.solver.means_rank, q_index, input_idx, nc)
    fdic["overlap"] = lambda q_index, input_idx, nc:call_method_weighted(test.solver.overlap_rank, q_index, input_idx, nc)
    fdic["quadtree"] = lambda q_index, input_idx, nc:call_method_weighted(test.solver.quadtree_rank, q_index,input_idx,  nc)
    fdic["flowtree"] = lambda q_index, input_idx, nc:call_method_weighted(test.solver.flowtree_rank, q_index, input_idx, nc)
    fdic["rwmd"] = lambda q_index, input_idx, nc:call_method_uniform(test.rwmd, q_index, input_idx, nc)
    fdic["lcwmd"] = lambda q_index, input_idx, nc:call_method_uniform(test.lc_wmd, q_index, input_idx, nc, method_param)
    fdic["sinkhorn"] = lambda q_index, input_idx, nc:call_method_uniform(test.sinkhorn, q_index, input_idx, nc, method_param)
    fdic["exact"] = lambda q_index, input_idx, nc:call_exact(q_index, input_idx)
    print('step 2')
    ### Main

    accuracy = 0
    accs = np.zeros(len(methods))
    total_time = 0
    clength = 0
    input_idx = None
    orig_input_idx = np.zeros(len(test.dataset), dtype=np.int32)
    for i in range(len(test.dataset)):
        orig_input_idx[i] = i

    start = time.time()
    for q in range(qn):
        input_idx = orig_input_idx
        for m in range(len(methods)):
            input_idx = fdic[methods[m]](q, input_idx, ncs[m])
            if m==0:
                clength += np.mean([len(test.dataset[j]) for j in input_idx])
            if answers[0,q] in input_idx:
                accs[m] += 1
        if answers[0,q] in input_idx:
            accuracy += 1
        total_time = time.time() - start
        # At the end of each iteration of the main loop, input_idx is a list of indices (in the dataset) of the final candidates
        # for the current query, of the length specified by the final number in the input list of candidate numbers (third parameter).
        # As an example let us print the top-10 nearest neighbors found by the pipeline for each of the first three queries:
        if q < 3:
            print("Nearest neighbors found by pipeline for query " + str(q) + ": " + str(input_idx))
        accuracies.append(accuracy)

    print("=== Output ===")
    print("Total time: ", total_time*1./qn)
    print("Final accuracy:", accuracy*1./qn)
    print("Accuracy after each stage of the pipeline:", [acc*1./qn for acc in accs])
print('total', sum(accuracies)/100)

generation step 0
generation step 1
generation step 2
generation step 3
generation step 4
generation step 5
generation step 6
step 1
step 2
Nearest neighbors found by pipeline for query 0: [6691]
=== Output ===
Total time:  0.2528350353240967
Final accuracy: 1.0
Accuracy after each stage of the pipeline: [1.0]
generation step 0
generation step 1
generation step 2
generation step 3
generation step 4
generation step 5
generation step 6
step 1
step 2
Nearest neighbors found by pipeline for query 0: [2479]
=== Output ===
Total time:  0.25827503204345703
Final accuracy: 1.0
Accuracy after each stage of the pipeline: [1.0]
generation step 0
generation step 1
generation step 2
generation step 3
generation step 4
generation step 5
generation step 6
step 1
step 2
Nearest neighbors found by pipeline for query 0: [10487]
=== Output ===
Total time:  0.25081610679626465
Final accuracy: 1.0
Accuracy after each stage of the pipeline: [1.0]
generation step 0
generation step 1
generation step 2
generat

KeyboardInterrupt: 